In [1]:
! pip install qiskit

In [2]:
! pip install qiskit-machine-learning

In [3]:
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from qiskit import *

# External imports
from pylab import cm
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.visualization import circuit_drawer
from qiskit.algorithms.optimizers import SPSA

from qiskit.circuit.library import PauliFeatureMap
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import TrainableFidelityQuantumKernel
from qiskit_machine_learning.kernels.algorithms import QuantumKernelTrainer
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.datasets import ad_hoc_data

from qiskit.utils import QuantumInstance
from qiskit_machine_learning.kernels import QuantumKernel

import random
import time
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

/tmp/ipykernel_60/3445415176.py:21: DeprecationWarning: ``qiskit.algorithms`` has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. The ``qiskit.algorithms`` import path is deprecated as of qiskit-terra 0.25.0 and will be removed no earlier than 3 months after the release date. Please run ``pip install qiskit_algorithms`` and use ``import qiskit_algorithms`` instead.
  from qiskit.algorithms.optimizers import SPSA


In [4]:
df = pd.read_csv('Thesis IBM Dataset.csv')
df.head()

,Date,Time,Temperature(C),Feel likes(C),Chance of Prec.(%),Humidity(%),Visibity(km),Pressure(hPa),State,Real State,Rain,Cloudy,Sunny,Fog
0,27-06-2023,08:00,27,32,21,87,3.0,1003,Cloudy,Cloudy,0.0,1,0,0
1,27-06-2023,09:00,27,32,22,86,9.0,1003,Cloudy,Cloudy,0.0,1,0,0
2,27-06-2023,10:00,28,34,23,90,3.0,1003,Cloudy,Rain,1.0,1,0,0
3,27-06-2023,11:00,28,33,30,85,9.0,1003,Isolated Thunderstroms,Rain,1.0,1,0,0
4,27-06-2023,12:00,27,32,53,90,9.0,1003,Rain,Sunny,0.0,0,1,0


In [5]:
df.drop(columns=['Date'], inplace=True)
#df.drop(columns=['Temperature(C)'], inplace=True)
df

,Time,Temperature(C),Feel likes(C),Chance of Prec.(%),Humidity(%),Visibity(km),Pressure(hPa),State,Real State,Rain,Cloudy,Sunny,Fog
0,08:00,27,32,21,87,3.0,1003,Cloudy,Cloudy,0.0,1,0,0
1,09:00,27,32,22,86,9.0,1003,Cloudy,Cloudy,0.0,1,0,0
2,10:00,28,34,23,90,3.0,1003,Cloudy,Rain,1.0,1,0,0
3,11:00,28,33,30,85,9.0,1003,Isolated Thunderstroms,Rain,1.0,1,0,0
4,12:00,27,32,53,90,9.0,1003,Rain,Sunny,0.0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4499,19:00,20,20,1,77,8.0,1014,Clear,Clear,0.0,0,1,0
4500,20:00,19,19,1,80,14.0,1015,Clear,Clear,0.0,0,1,0
4501,21:00,19,19,2,82,14.0,1015,Clear,Clear,0.0,0,1,0
4502,22:00,18,18,2,83,14.0,1015,Clear,Clear,0.0,0,1,0


In [6]:
df.dropna(inplace=True)
df

,Time,Temperature(C),Feel likes(C),Chance of Prec.(%),Humidity(%),Visibity(km),Pressure(hPa),State,Real State,Rain,Cloudy,Sunny,Fog
0,08:00,27,32,21,87,3.0,1003,Cloudy,Cloudy,0.0,1,0,0
1,09:00,27,32,22,86,9.0,1003,Cloudy,Cloudy,0.0,1,0,0
2,10:00,28,34,23,90,3.0,1003,Cloudy,Rain,1.0,1,0,0
3,11:00,28,33,30,85,9.0,1003,Isolated Thunderstroms,Rain,1.0,1,0,0
4,12:00,27,32,53,90,9.0,1003,Rain,Sunny,0.0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4499,19:00,20,20,1,77,8.0,1014,Clear,Clear,0.0,0,1,0
4500,20:00,19,19,1,80,14.0,1015,Clear,Clear,0.0,0,1,0
4501,21:00,19,19,2,82,14.0,1015,Clear,Clear,0.0,0,1,0
4502,22:00,18,18,2,83,14.0,1015,Clear,Clear,0.0,0,1,0


In [7]:
df.drop(columns=['Time', 'State','Real State'], inplace=True)
df

,Temperature(C),Feel likes(C),Chance of Prec.(%),Humidity(%),Visibity(km),Pressure(hPa),Rain,Cloudy,Sunny,Fog
0,27,32,21,87,3.0,1003,0.0,1,0,0
1,27,32,22,86,9.0,1003,0.0,1,0,0
2,28,34,23,90,3.0,1003,1.0,1,0,0
3,28,33,30,85,9.0,1003,1.0,1,0,0
4,27,32,53,90,9.0,1003,0.0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
4499,20,20,1,77,8.0,1014,0.0,0,1,0
4500,19,19,1,80,14.0,1015,0.0,0,1,0
4501,19,19,2,82,14.0,1015,0.0,0,1,0
4502,18,18,2,83,14.0,1015,0.0,0,1,0


In [8]:

for col in df.columns:
    if (col == 'Rain'):
       
        df[col] = np.where(df[col] == 0.0, 0, 1)
df

,Temperature(C),Feel likes(C),Chance of Prec.(%),Humidity(%),Visibity(km),Pressure(hPa),Rain,Cloudy,Sunny,Fog
0,27,32,21,87,3.0,1003,0,1,0,0
1,27,32,22,86,9.0,1003,0,1,0,0
2,28,34,23,90,3.0,1003,1,1,0,0
3,28,33,30,85,9.0,1003,1,1,0,0
4,27,32,53,90,9.0,1003,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
4499,20,20,1,77,8.0,1014,0,0,1,0
4500,19,19,1,80,14.0,1015,0,0,1,0
4501,19,19,2,82,14.0,1015,0,0,1,0
4502,18,18,2,83,14.0,1015,0,0,1,0


In [9]:
from qiskit import *

In [10]:
IBMQ.load_account()

/tmp/ipykernel_60/485379158.py:1: DeprecationWarning: The package qiskit.providers.ibmq is being deprecated. Please see https://ibm.biz/provider_migration_guide to get instructions on how to migrate to qiskit-ibm-provider (https://github.com/Qiskit/qiskit-ibm-provider) and qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  IBMQ.load_account()
/tmp/ipykernel_60/485379158.py:1: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_ibm_provider.IBMProvider'
  IBMQ.load_account()


<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [11]:
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_extended_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibm_brisbane') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibm_kyoto') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibm_osaka') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [12]:
print("Using 3 features Price, Open, High: ")
sz = []
svm_acc = []
qsvm_acc = []
for data_size in range(50, 300, 50):
    print("for size", data_size)
    sz.append(data_size)
    st = random.randint(0, 300-data_size)
    x = df.iloc[st:st+data_size, 0:-1]
    y = df.iloc[st:st+data_size, -1]
    # spltting the dataset into train and test set
    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 31)
    from sklearn import svm
    svc_clf = svm.SVC(random_state = 7)
    svc_clf.fit(x_train, y_train)
    svc_score = svc_clf.score(x_test, y_test)
    print(f"svm classification score: {svc_score}")
    svm_acc.append(svc_score)
    import qiskit
    import matplotlib.pyplot as plt
    import numpy as np

    from sklearn.svm import SVC
    from sklearn.cluster import SpectralClustering
    from sklearn.metrics import normalized_mutual_info_score

    from qiskit import BasicAer
    from qiskit.circuit.library import ZZFeatureMap
    from qiskit.utils import QuantumInstance, algorithm_globals
    from qiskit_machine_learning.algorithms import QSVC
    from qiskit_machine_learning.kernels import QuantumKernel
    from qiskit_machine_learning.datasets import ad_hoc_data

    seed = 12345
    algorithm_globals.random_seed = seed
    # Building Kernel
    dse_feature_map = ZZFeatureMap(feature_dimension=10, reps=2, entanglement="linear")

    dse_backend = QuantumInstance(
        BasicAer.get_backend('qasm_simulator'), shots=1024, seed_simulator=seed, seed_transpiler=seed
    )

    dse_kernel = QuantumKernel(feature_map=dse_feature_map, quantum_instance=dse_backend)
    dse_svc = SVC(kernel=dse_kernel.evaluate)
    dse_svc.fit(x_train, y_train)
    dse_score = dse_svc.score(x_test, y_test)

    print(f"qsvm classification test score: {dse_score}")
    qsvm_acc.append(dse_score)

Using 3 features Price, Open, High: 
for size 50
Traceback (most recent call last):
  Cell In[12], line 16
    svc_clf.fit(x_train, y_train)
  File /opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:201 in fit
    y = self._validate_targets(y)
  File /opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:749 in _validate_targets
    raise ValueError(
ValueError: The number of classes has to be greater than one; got 1 class

Use %tb to get the full traceback.


In [ ]:
dse_score

In [ ]:
df

In [ ]:
print(svm_acc)
print(qsvm_acc)

In [ ]:
sz = [50,100,150,200]

import random
total = 5
for cnt in range(total):
    print("current iteration: ", cnt+1)
    for idx in range(len(sz)):
        data_size = sz[idx]
        print("for size", data_size)
        st = random.randint(0, 200-data_size)
        x = df.iloc[st:st+data_size, 0:-1]
        y = df.iloc[st:st+data_size, -1]
        # spltting the dataset into train and test set
        from sklearn.model_selection import train_test_split
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 31)
        from sklearn import svm
        svc_clf = svm.SVC(random_state = 7)
        svc_clf.fit(x_train, y_train)
        svc_score = svc_clf.score(x_test, y_test)
        print(f"svm classification score: {svc_score}")
        svm_acc[idx] = svm_acc[idx] + svc_score
        import qiskit
        import matplotlib.pyplot as plt
        import numpy as np

        from sklearn.svm import SVC
        from sklearn.cluster import SpectralClustering
        from sklearn.metrics import normalized_mutual_info_score

        from qiskit import BasicAer
        from qiskit.circuit.library import ZZFeatureMap
        from qiskit.utils import QuantumInstance, algorithm_globals
        from qiskit_machine_learning.algorithms import QSVC
        from qiskit_machine_learning.kernels import QuantumKernel
        from qiskit_machine_learning.datasets import ad_hoc_data

        seed = 12345
        algorithm_globals.random_seed = seed
        # Building Kernel
        dse_feature_map = ZZFeatureMap(feature_dimension=7, reps=2, entanglement="linear")

        dse_backend = QuantumInstance(
            BasicAer.get_backend('qasm_simulator'), shots=1024, seed_simulator=seed, seed_transpiler=seed
        )

        dse_kernel = QuantumKernel(feature_map=dse_feature_map, quantum_instance=dse_backend)
        dse_svc = SVC(kernel=dse_kernel.evaluate)
        dse_svc.fit(x_train, y_train)
        dse_score = dse_svc.score(x_test, y_test)

        print(f"qsvm classification test score: {dse_score}")
        qsvm_acc[idx] = qsvm_acc[idx] + dse_score
    print("=============================================")

In [ ]:
print(svm_acc, qsvm_acc)

In [ ]:
for idx in range(len(sz)):
    print(idx, svm_acc[idx]/2, '\t\t', qsvm_acc[idx]/2)